In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets
from ipywidgets import interact, interactive

In [2]:
# output directory
direction = "down"
species = "HR"
now = datetime.datetime.now()
line_enrichment = f'../data/line_enrichment_{direction}_{species}_{now.strftime("%y%m")}'
os.makedirs(line_enrichment, exist_ok=True)

In [3]:
def count_goslim(df, file_name: str):
    unique_gene_pairs = df.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_human', 'Gene stable ID_rice'])
    goslim_counts = unique_gene_pairs.groupby(['GOSlim GOA Accession(s)', 'GOSlim GOA Description', 'GOSlim_domain']).size().reset_index(name='counts')
    goslim_counts = goslim_counts.sort_values(by='counts', ascending=False).copy()
    goslim_counts.reset_index(drop=True, inplace=True)
    results_directory = f'../data/{line_enrichment}/goslim_correspondence_counts_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    goslim_counts.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)
    return goslim_counts

In [4]:
merged_df_goslim_pd = pd.read_csv('../data/GOSlim_merge_HR/GOslim_merge_common_goslim_correspondence_all.tsv', 
                                  sep='\t',
                                  low_memory=False)
merged_df_up = pd.read_csv(f'../data/circos_{direction}_{species}_2311/combined_goslim_down.tsv', sep='\t')

display(merged_df_goslim_pd, merged_df_up)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,Gene stable ID_human,Gene stable ID_rice
0,GO:0005886,plasma membrane,cellular_component,Os01g0100400,ENSG00000186092
1,GO:0005886,plasma membrane,cellular_component,Os01g0103600,ENSG00000186092
2,GO:0005886,plasma membrane,cellular_component,Os01g0104000,ENSG00000186092
3,GO:0005886,plasma membrane,cellular_component,Os01g0108000,ENSG00000186092
4,GO:0005886,plasma membrane,cellular_component,Os01g0110100,ENSG00000186092
...,...,...,...,...,...
205962886,GO:0045182,translation regulator activity,molecular_function,Os12g0507200,ENSG00000205916
205962887,GO:0045182,translation regulator activity,molecular_function,Os12g0541500,ENSG00000205916
205962888,GO:0045182,translation regulator activity,molecular_function,Os12g0607100,ENSG00000205916
205962889,GO:0045182,translation regulator activity,molecular_function,Os12g0617100,ENSG00000205916


,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,Gene stable ID_rice,Gene stable ID_human,Chromosome/scaffold name_rice,start1 (bp),end1 (bp),Chromosome/scaffold name_human,start2 (bp),end2 (bp),HN-score(HN5)_rice,HN-score(HN5)_human
0,GO:0003824,catalytic activity,molecular_function,Os02g0504000,ENSG00000142731,R_2,17880882,17886285,H_4,127880893,127899224,-84,-17
1,GO:0003824,catalytic activity,molecular_function,Os02g0121700,ENSG00000151117,R_2,1145837,1151052,H_11,18693122,18704785,-59,-16
2,GO:0003824,catalytic activity,molecular_function,Os02g0571100,ENSG00000151117,R_2,21841092,21847545,H_11,18693122,18704785,-53,-16
3,GO:0003824,catalytic activity,molecular_function,Os05g0360400,ENSG00000151117,R_5,17175702,17176974,H_11,18693122,18704785,-54,-16
4,GO:0003824,catalytic activity,molecular_function,Os11g0641500,ENSG00000151117,R_11,25423701,25426045,H_11,18693122,18704785,-54,-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18457,GO:0006091,generation of precursor metabolites and energy,biological_process,Os06g0718400,ENSG00000213689,R_6,30528045,30529070,H_3,48465811,48467645,-89,-28
18458,GO:0030312,external encapsulating structure,cellular_component,Os03g0651800,ENSG00000111186,R_3,25315697,25317065,H_12,1529891,1647212,-68,-17
18459,GO:0005635,nuclear envelope,cellular_component,Os12g0236100,ENSG00000213689,R_12,7441389,7446142,H_3,48465811,48467645,-81,-28
18460,GO:0005635,nuclear envelope,cellular_component,Os12g0236100,ENSG00000205808,R_12,7441389,7446142,H_9,4662294,4665258,-81,-18


In [5]:
gosilm_correspondence_up_counts = count_goslim(merged_df_up, f'goslim_correspondence_counts_{direction}.tsv')
goslim_correspondence_all_counts = count_goslim(merged_df_goslim_pd, 'goslim_correspondence_counts_all.tsv')

display(gosilm_correspondence_up_counts, goslim_correspondence_all_counts)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts
0,GO:0003824,catalytic activity,molecular_function,7998
1,GO:0005634,nucleus,cellular_component,2790
2,GO:0005886,plasma membrane,cellular_component,1230
3,GO:0016787,hydrolase activity,molecular_function,1107
4,GO:0016740,transferase activity,molecular_function,1080
5,GO:0005576,extracellular region,cellular_component,870
6,GO:0003677,DNA binding,molecular_function,672
7,GO:0048856,anatomical structure development,biological_process,590
8,GO:0005829,cytosol,cellular_component,567
9,GO:0036211,protein modification process,biological_process,555


,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts
0,GO:0003824,catalytic activity,molecular_function,62834292
1,GO:0005634,nucleus,cellular_component,50586355
2,GO:0005886,plasma membrane,cellular_component,14121523
3,GO:0036211,protein modification process,biological_process,11265607
4,GO:0016740,transferase activity,molecular_function,10856023
5,GO:0016787,hydrolase activity,molecular_function,8674587
6,GO:0005829,cytosol,cellular_component,7811118
7,GO:0048856,anatomical structure development,biological_process,7216908
8,GO:0003677,DNA binding,molecular_function,5819761
9,GO:0003723,RNA binding,molecular_function,4482192


In [6]:
# for debugging purpose
total_counts_up = gosilm_correspondence_up_counts['counts'].sum()
total_counts_all = goslim_correspondence_all_counts['counts'].sum()

unique_rows_up = merged_df_up.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_human', 'Gene stable ID_rice']).shape[0]
unique_rows_all = merged_df_goslim_pd.drop_duplicates(subset=['GOSlim GOA Accession(s)', 'Gene stable ID_human', 'Gene stable ID_rice']).shape[0]

print(f"Total counts: {unique_rows_all}")
print(f"up counts: {unique_rows_up}")
print(f"Counts match rows: {unique_rows_up == total_counts_up}")

Total counts: 205962891
up counts: 18462
Counts match rows: True


In [7]:
# fold enrichment
def fold_enrichment(df_goslim_up_counts, df_goslim_all_counts, unique_genes_up, unique_genes_all, file_name: str):
    """_summary_
    Args:
        df_goslim_up_counts (dataframe): _description_
        df_goslim_all_counts (dataframe): _description_
        unique_genes_up (): _description_
        unique_genes_all (dataframe): _description_
        file_name (str): _description_

    Returns:
        dataframe : _description_
    """
    merged_df = pd.merge(df_goslim_up_counts, 
                         df_goslim_all_counts,
                         on=['GOSlim GOA Accession(s)', 'GOSlim GOA Description', 'GOSlim_domain'], 
                         how='right', # if there is no match, fill with NaN or 0
                         suffixes=('_down', '_all'))
    
    merged_df['counts_down'] = merged_df['counts_down'].fillna(0)
    merged_df['counts_all'] = merged_df['counts_all'].fillna(0)
    merged_df['down_ratio'] = merged_df['counts_down'] / unique_genes_up
    merged_df['all_ratio'] = merged_df['counts_all'] / unique_genes_all
    merged_df['fold_enrichment'] = merged_df['down_ratio'] / merged_df['all_ratio']
    merged_df.replace([np.inf, -np.inf], np.nan, inplace=True) # inf -> nan
    results_directory = f'../data/{line_enrichment}/goslim_fold_enrichment_correspondence_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    merged_df.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)
    
    return merged_df


goslim_correspondence_enrichment = fold_enrichment(gosilm_correspondence_up_counts, 
                                                   goslim_correspondence_all_counts, 
                                                   unique_rows_up, 
                                                   unique_rows_all, 
                                                   f'goslim_correspondence_fold_enrichment_{direction}.tsv')
display(goslim_correspondence_enrichment)

,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts_down,counts_all,down_ratio,all_ratio,fold_enrichment
0,GO:0003824,catalytic activity,molecular_function,7998.0,62834292,0.433214,0.305076,1.420021
1,GO:0005634,nucleus,cellular_component,2790.0,50586355,0.151121,0.245609,0.615292
2,GO:0005886,plasma membrane,cellular_component,1230.0,14121523,0.066623,0.068563,0.971704
3,GO:0036211,protein modification process,biological_process,555.0,11265607,0.030062,0.054697,0.549602
4,GO:0016740,transferase activity,molecular_function,1080.0,10856023,0.058499,0.052709,1.109847
5,GO:0016787,hydrolase activity,molecular_function,1107.0,8674587,0.059961,0.042117,1.423669
6,GO:0005829,cytosol,cellular_component,567.0,7811118,0.030712,0.037925,0.809804
7,GO:0048856,anatomical structure development,biological_process,590.0,7216908,0.031958,0.035040,0.912034
8,GO:0003677,DNA binding,molecular_function,672.0,5819761,0.036399,0.028256,1.288173
9,GO:0003723,RNA binding,molecular_function,0.0,4482192,0.000000,0.021762,0.000000


## Q-value

In [9]:
def calculate_p_q_values(df_enrichment, unique_correspondence_down, unique_correspondence_all, file_name: str):
    p_values = []
    df_enrichment[f'counts_{direction}'] = df_enrichment[f'counts_{direction}'].astype(int) # float(0.0) to int(0)
    for index, row in df_enrichment.iterrows():
        if row[f'counts_{direction}'] == 0:
            p_values.append(1.0)
        else:
            observed_down_correspondence = row[f'counts_{direction}'] # create contingency table
            observed_all_correspondence = row['counts_all']
            observed_not_down_correspondence = observed_all_correspondence - observed_down_correspondence
            total_not_down_correspondence = unique_correspondence_all - unique_correspondence_down
            table = [
            [observed_down_correspondence, unique_correspondence_down - observed_down_correspondence],
            [observed_not_down_correspondence, total_not_down_correspondence - observed_not_down_correspondence]
            ]
            _, p_value = fisher_exact(table, alternative='greater') # Fisher's exact test
            p_values.append(p_value)
    
    _, q_values, _, _ = multipletests(p_values, 
                                      alpha=0.05, 
                                      method='fdr_bh') # Benjamini/Hochberg method
    
    df_enrichment['p_value'] = p_values
    df_enrichment['q_value'] = q_values

    results_directory = f'../data/{line_enrichment}/goslim_correspondence_q_values_{direction}_{species}'
    os.makedirs(results_directory, exist_ok=True)
    df_enrichment['GOSlim'] = df_enrichment['GOSlim GOA Accession(s)'] + ": " + df_enrichment['GOSlim GOA Description']
    cols = df_enrichment.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df_enrichment = df_enrichment[cols]
    df_enrichment.drop(['GOSlim GOA Accession(s)', 'GOSlim GOA Description'], axis=1)
    df_enrichment = df_enrichment[df_enrichment[f'counts_{direction}'] != 0]
    df_enrichment = df_enrichment.sort_values(by='fold_enrichment', ascending=False)
    df_enrichment.reset_index(drop=True, inplace=True)
    df_enrichment.to_csv(f'{results_directory}/{file_name}', sep='\t', index=False)

    return df_enrichment

goslim_correspondence_with_p_q = calculate_p_q_values(goslim_correspondence_enrichment.copy(),
                                                      unique_rows_up,
                                                      unique_rows_all, 
                                                      f'goslim_correspondence_fold_enrichment_p_q_{direction}.tsv')


display(goslim_correspondence_with_p_q)

,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,counts_down,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value
0,GO:0005764: lysosome,GO:0005764,lysosome,cellular_component,21,45237,0.001137,0.000220,5.178879,2.405986e-09,1.168622e-08
1,GO:0008289: lipid binding,GO:0008289,lipid binding,molecular_function,64,257535,0.003467,0.001250,2.772388,1.956956e-12,1.330730e-11
2,GO:0005975: carbohydrate metabolic process,GO:0005975,carbohydrate metabolic process,biological_process,153,676139,0.008287,0.003283,2.524444,1.642043e-23,1.395737e-22
3,GO:0005576: extracellular region,GO:0005576,extracellular region,cellular_component,870,3845282,0.047124,0.018670,2.524069,2.046157e-127,3.478467e-126
4,GO:0006629: lipid metabolic process,GO:0006629,lipid metabolic process,biological_process,204,1517670,0.011050,0.007369,1.499557,2.974814e-08,1.264296e-07
5,GO:0005773: vacuole,GO:0005773,vacuole,cellular_component,54,422166,0.002925,0.002050,1.426989,7.697182e-03,2.617042e-02
6,GO:0016787: hydrolase activity,GO:0016787,hydrolase activity,molecular_function,1107,8674587,0.059961,0.042117,1.423669,3.160887e-30,3.582339e-29
7,GO:0003824: catalytic activity,GO:0003824,catalytic activity,molecular_function,7998,62834292,0.433214,0.305076,1.420021,2.079914e-294,7.071707e-293
8,GO:0003677: DNA binding,GO:0003677,DNA binding,molecular_function,672,5819761,0.036399,0.028256,1.288173,8.590193e-11,4.867776e-10
9,GO:0016740: transferase activity,GO:0016740,transferase activity,molecular_function,1080,10856023,0.058499,0.052709,1.109847,2.787134e-04,1.052917e-03


In [10]:
#data preprocessing for plotting
goslim_correspondence_with_p_q.rename(columns={f'counts_{direction}': 'line counts'}, inplace=True)

if 'p_value' in goslim_correspondence_with_p_q.columns:
    goslim_correspondence_with_p_q['-log10(p-value)'] = -np.log10(goslim_correspondence_with_p_q['p_value'])
    
if 'q_value' in goslim_correspondence_with_p_q.columns:
    goslim_correspondence_with_p_q['-log10(q-value)'] = -np.log10(goslim_correspondence_with_p_q['q_value'])


cc = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'cellular_component']
bp = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'biological_process']
mf = goslim_correspondence_with_p_q.loc[goslim_correspondence_with_p_q['GOSlim_domain'] == 'molecular_function']

cc.reset_index(drop=True, inplace=True)
bp.reset_index(drop=True, inplace=True)
mf.reset_index(drop=True, inplace=True)

display(cc, bp, mf)

,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0005764: lysosome,GO:0005764,lysosome,cellular_component,21,45237,0.001137,0.000220,5.178879,2.405986e-09,1.168622e-08,8.618707e+00,7.932326
1,GO:0005576: extracellular region,GO:0005576,extracellular region,cellular_component,870,3845282,0.047124,0.018670,2.524069,2.046157e-127,3.478467e-126,1.266891e+02,125.458612
2,GO:0005773: vacuole,GO:0005773,vacuole,cellular_component,54,422166,0.002925,0.002050,1.426989,7.697182e-03,2.617042e-02,2.113668e+00,1.582189
3,GO:0005886: plasma membrane,GO:0005886,plasma membrane,cellular_component,1230,14121523,0.066623,0.068563,0.971704,8.550350e-01,1.000000e+00,6.801609e-02,-0.000000
4,GO:0005829: cytosol,GO:0005829,cytosol,cellular_component,567,7811118,0.030712,0.037925,0.809804,9.999999e-01,1.000000e+00,2.305136e-08,-0.000000
5,GO:0005635: nuclear envelope,GO:0005635,nuclear envelope,cellular_component,3,50592,0.000162,0.000246,0.661530,8.303609e-01,1.000000e+00,8.073310e-02,-0.000000
6,GO:0030312: external encapsulating structure,GO:0030312,external encapsulating structure,cellular_component,12,203467,0.000650,0.000988,0.657957,9.507781e-01,1.000000e+00,2.192081e-02,-0.000000
7,GO:0005634: nucleus,GO:0005634,nucleus,cellular_component,2790,50586355,0.151121,0.245609,0.615292,1.000000e+00,1.000000e+00,-0.000000e+00,-0.000000
8,GO:0005856: cytoskeleton,GO:0005856,cytoskeleton,cellular_component,24,725760,0.001300,0.003524,0.368917,1.000000e+00,1.000000e+00,6.921371e-10,-0.000000
9,GO:0005739: mitochondrion,GO:0005739,mitochondrion,cellular_component,22,1951236,0.001192,0.009474,0.125783,1.000000e+00,1.000000e+00,-0.000000e+00,-0.000000


,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0005975: carbohydrate metabolic process,GO:0005975,carbohydrate metabolic process,biological_process,153,676139,0.008287,0.003283,2.524444,1.642043e-23,1.395737e-22,22.784615,21.855197
1,GO:0006629: lipid metabolic process,GO:0006629,lipid metabolic process,biological_process,204,1517670,0.011050,0.007369,1.499557,2.974814e-08,1.264296e-07,7.526540,6.898151
2,GO:0048856: anatomical structure development,GO:0048856,anatomical structure development,biological_process,590,7216908,0.031958,0.035040,0.912034,9.899762e-01,1.000000e+00,0.004375,-0.000000
3,GO:0030154: cell differentiation,GO:0030154,cell differentiation,biological_process,94,1272360,0.005092,0.006178,0.824191,9.759711e-01,1.000000e+00,0.010563,-0.000000
4,GO:0036211: protein modification process,GO:0036211,protein modification process,biological_process,555,11265607,0.030062,0.054697,0.549602,1.000000e+00,1.000000e+00,-0.000000,-0.000000
5,GO:0006091: generation of precursor metabolite...,GO:0006091,generation of precursor metabolites and energy,biological_process,12,265631,0.000650,0.001290,0.503979,9.972028e-01,1.000000e+00,0.001217,-0.000000


,GOSlim,GOSlim GOA Accession(s),GOSlim GOA Description,GOSlim_domain,line counts,counts_all,down_ratio,all_ratio,fold_enrichment,p_value,q_value,-log10(p-value),-log10(q-value)
0,GO:0008289: lipid binding,GO:0008289,lipid binding,molecular_function,64,257535,0.003467,0.001250,2.772388,1.956956e-12,1.330730e-11,1.170842e+01,10.875910
1,GO:0016787: hydrolase activity,GO:0016787,hydrolase activity,molecular_function,1107,8674587,0.059961,0.042117,1.423669,3.160887e-30,3.582339e-29,2.950019e+01,28.445833
2,GO:0003824: catalytic activity,GO:0003824,catalytic activity,molecular_function,7998,62834292,0.433214,0.305076,1.420021,2.079914e-294,7.071707e-293,2.936820e+02,292.150476
3,GO:0003677: DNA binding,GO:0003677,DNA binding,molecular_function,672,5819761,0.036399,0.028256,1.288173,8.590193e-11,4.867776e-10,1.006600e+01,9.312669
4,GO:0016740: transferase activity,GO:0016740,transferase activity,molecular_function,1080,10856023,0.058499,0.052709,1.109847,2.787134e-04,1.052917e-03,3.554842e+00,2.977606
5,GO:0140110: transcription regulator activity,GO:0140110,transcription regulator activity,molecular_function,200,2256826,0.010833,0.010957,0.988649,5.740091e-01,1.000000e+00,2.410812e-01,-0.000000
6,GO:0005215: transporter activity,GO:0005215,transporter activity,molecular_function,84,1929898,0.004550,0.009370,0.485574,1.000000e+00,1.000000e+00,7.039591e-15,-0.000000
7,GO:0005198: structural molecule activity,GO:0005198,structural molecule activity,molecular_function,28,650310,0.001517,0.003157,0.480339,9.999963e-01,1.000000e+00,1.598255e-06,-0.000000


In [12]:
# enrichment result plot function
current_fig = None # global variable

def plot_goslim_dotplot_with_lines(data, col1, goslim_col, col2, col3, ylabel_suffix='', width=6, height=9, palette="flare"):

    # 1. figure size
    global current_fig
    figsize = (width, height)
    current_fig, ax = plt.subplots(figsize=figsize)

    # 2. color normalization
    norm = mcolors.Normalize(vmin=data[col3].min(), vmax=data[col3].max())

    # 3.color palette
    color_palette = sns.color_palette(palette, as_cmap=True)

    # 4.plotting
    sns.scatterplot(
        data=data,
        x=col1,
        y=goslim_col,
        size=col2, 
        hue=col3,     
        palette=palette, 
        legend='brief',
        ax=ax
    )
    # 5.lines 
    for index, row in data.iterrows():
        ax.plot([0, row[col1]], [row[goslim_col], row[goslim_col]], color=color_palette(norm(row[col3])), lw=1)
    
    ax.grid(color='b', linestyle=':', linewidth=0.1)
    ax.set_xlabel(col1)
    ylabel_text = 'Common GOSlim Terms' + ('' + ylabel_suffix  if ylabel_suffix else '' )
    ax.set_ylabel(ylabel_text)

    plt.show()

# execute only this function to plot
# plot_goslim_dotplot_with_lines(goslim_correspondence_with_p_q.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix='') # add copy() to avoid SettingWithCopyWarning
# plot_goslim_dotplot_with_lines(cc.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Cellular Component)')
# plot_goslim_dotplot_with_lines(bp.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Biological Process)')
# plot_goslim_dotplot_with_lines(mf.copy(), 'fold_enrichment', 'GOSlim', 'line counts', 'q_value', ylabel_suffix=' (Molecular Function)')

In [13]:
# interactive setting
filename_text = widgets.Text(value='', description='File Name:')
png_directory = f'../data/{line_enrichment}/png'
os.makedirs(png_directory, exist_ok=True)

def save_plot(button):
    global current_fig
    if current_fig:
        filename = filename_text.value
        if filename:  
            current_fig.savefig(f'{png_directory}/{filename}.png', dpi=800, bbox_inches='tight')
        else:
            current_fig.savefig(f'{png_directory}/enrichment_result.png', dpi=800, bbox_inches='tight')


save_button = widgets.Button(description="Save Plot")
save_button.on_click(save_plot)

def interactive_plot(data):
    w = interactive(
        plot_goslim_dotplot_with_lines,
        data=widgets.fixed(data),
        col1=widgets.Dropdown(options= [('fold enrichment', 'fold_enrichment'), 
                                        ('-log10(p-value)', '-log10(p-value)'), 
                                        ('-log10(q-value)', '-log10(q-value)')], 
                                        value='fold_enrichment', 
                                        description='x-axis:'),
        goslim_col=widgets.fixed('GOSlim'),
        col2=widgets.Dropdown(options=data.columns.tolist(), value='line counts', description='size:'),
        col3=widgets.Dropdown(options= [('fold enrichment', 'fold_enrichment'), 
                                        ('-log10(p-value)', '-log10(p-value)'), 
                                        ('-log10(q-value)', '-log10(q-value)')], 
                                        value='-log10(q-value)', 
                                        description='hue:'),
        ylabel_suffix=widgets.Dropdown(
            options=[('', ''), 
                     ('cc', '(Cellular Component)'), 
                     ('bp', '(Biological Process)'), 
                     ('mf', '(Molecular Function)')],
            value='',
            description='y-label suffix:'),
        width=widgets.IntSlider(min=1, max=30, step=1, value=6, description='width:', readout_format='0.0f'),
        height=widgets.IntSlider(min=1, max=30, step=1, value=9, description='height:', readout_format='0.0f'),
        palette=widgets.Dropdown(options=['flare', 'magma', 'viridis'], value='flare', description='color palette:')
        
    )
    display(w, filename_text, save_button)

In [14]:
interactive_plot(goslim_correspondence_with_p_q.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [15]:
interactive_plot(cc.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='enrichment_result_down_all', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [16]:
interactive_plot(bp.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='enrichment_result_down_cc', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())

In [17]:
interactive_plot(mf.copy())

interactive(children=(Dropdown(description='x-axis:', options=(('fold enrichment', 'fold_enrichment'), ('-log1…

Text(value='enrichment_result_down_bp', description='File Name:')

Button(description='Save Plot', style=ButtonStyle())